<cell_type>markdown</cell_type># minbpe: Minimal Byte Pair Encoding Tokenizer

This notebook contains a faithful implementation of Andrej Karpathy's minbpe tokenizer, emphasizing:
- Clean, minimal code
- Byte-level tokenization
- Educational clarity

The implementation follows the core design philosophy of Karpathy's approach, preserving the simplicity and readability of the original while adding several enhancements:

## Key Features
- **Basic Tokenizer**: Core BPE algorithm implementation
- **RegexTokenizer**: Enhanced with GPT2/GPT4 pattern splitting
- **SpecialTokensTokenizer**: Support for special tokens with flexible handling options
- **GPT4Tokenizer**: Compatible with OpenAI's tokenizers with byte shuffling
- **Save/Load**: Compatible with Karpathy's .model/.vocab format

Each feature is extensively documented and tested to provide a complete educational resource for understanding modern tokenization approaches.

In [1]:
import os
import json
import time
import regex as re
from collections import Counter
from typing import List, Dict, Tuple, Optional, Set, Any, Union

## Base Tokenizer Implementation

We start with a basic tokenizer class that implements the core BPE algorithm without any regex-based preprocessing.

In [2]:
class Tokenizer:
    """A minimal Byte Pair Encoding tokenizer implementation."""
    
    def __init__(self):
        # Initialize with the base 256 tokens (raw bytes 0-255)
        self.merges = {}  # (token1, token2) -> new_token_id 
        self.vocab = {}   # token_id -> token (bytes)
        self.vocab_size = 0
        self.special_tokens = {}
        
        # Pre-populate the vocabulary with the basic 256 byte tokens
        for i in range(256):
            token = bytes([i])
            self.vocab[i] = token
            
        self.vocab_size = 256
    
    def train(self, text: str, vocab_size: int, verbose: bool = False) -> None:
        """Train the tokenizer on text, extending the vocabulary to the desired size."""
        # Convert text to bytes
        text_bytes = text.encode("utf-8")
        ids = list(text_bytes)
        
        # Keep track of progress
        if verbose:
            print(f"Training BPE tokenizer to vocab size {vocab_size}")
            print(f"Text size: {len(text)} chars, {len(ids)} bytes")
        
        # Iteratively merge the most frequent pair until we reach the desired vocab size
        num_merges = vocab_size - 256
        for i in range(num_merges):
            # Count frequencies of adjacent pairs
            stats = self.get_stats(ids)
            if not stats:
                break
                
            # Find the most frequent pair
            pair = max(stats, key=stats.get)
            
            # Create a new token for this pair
            token1, token2 = pair
            new_token = self.vocab[token1] + self.vocab[token2]
            new_id = self.vocab_size
            
            # Add merge to our vocabulary
            self.merges[pair] = new_id
            self.vocab[new_id] = new_token
            self.vocab_size += 1
            
            # Apply the merge to the current token list
            ids = self.merge(ids, pair, new_id)
            
            # Print progress
            if verbose and i % 100 == 0:
                print(f"Merge #{i}: pair {pair} -> {new_id}, corpus now {len(ids)} tokens")
    
    def get_stats(self, ids: List[int]) -> Dict[Tuple[int, int], int]:
        """Count the frequencies of adjacent token pairs."""
        stats = Counter()
        for i in range(len(ids) - 1):
            pair = (ids[i], ids[i+1])
            stats[pair] += 1
        return stats
    
    def merge(self, ids: List[int], pair: Tuple[int, int], new_id: int) -> List[int]:
        """Replace all occurrences of a token pair with a new token ID."""
        # Create a new list for the merged result
        new_ids = []
        i = 0
        while i < len(ids):
            # If we're at the last token, just add it
            if i == len(ids) - 1:
                new_ids.append(ids[i])
                break
            
            # If current pair matches, merge and add the new token
            if ids[i] == pair[0] and ids[i+1] == pair[1]:
                new_ids.append(new_id)
                i += 2  # Skip both tokens
            else:
                new_ids.append(ids[i])
                i += 1  # Move to next token
        
        return new_ids
    
    def encode(self, text: str) -> List[int]:
        """Encode text to token IDs."""
        # Convert text to bytes
        text_bytes = text.encode("utf-8")
        ids = list(text_bytes)
        
        # Apply merges iteratively, in the order they were learned
        while len(ids) >= 2:
            # Find valid merge pairs in the current sequence
            pairs = [(ids[i], ids[i+1]) for i in range(len(ids)-1)]
            valid_pairs = [(pair, self.merges[pair]) for pair in pairs if pair in self.merges]
            
            # If no valid pairs, we're done
            if not valid_pairs:
                break
                
            # Find the pair with the lowest merge ID (first learned)
            pair, new_id = min(valid_pairs, key=lambda x: x[1])
            
            # Apply the merge
            ids = self.merge(ids, pair, new_id)
        
        return ids
    
    def decode(self, ids: List[int]) -> str:
        """Decode token IDs back to text."""
        # Convert token IDs to bytes
        bytes_list = []
        for token_id in ids:
            bytes_list.extend(self.vocab[token_id])
        
        # Convert bytes to UTF-8 text
        text = bytes(bytes_list).decode("utf-8", errors="replace")
        return text
    
    def save(self, file_path: str) -> None:
        """Save the tokenizer to a file."""
        # Prepare model data - convert bytes to lists for JSON serialization
        model_data = {
            "vocab_size": self.vocab_size,
            "merges": {f"{t1},{t2}": idx for (t1, t2), idx in self.merges.items()},
            "vocab": {str(i): list(t) for i, t in self.vocab.items() if i >= 256},
            "special_tokens": self.special_tokens
        }
        
        # Save to file
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(model_data, f, ensure_ascii=False, indent=2)
    
    def load(self, file_path: str) -> None:
        """Load a tokenizer from a file."""
        # Read the model data
        with open(file_path, 'r', encoding='utf-8') as f:
            model_data = json.load(f)
        
        # Reset the tokenizer
        self.__init__()
        
        # Load the vocabulary
        self.vocab_size = model_data["vocab_size"]
        
        # Add vocabulary items (skipping the base 256 bytes already initialized)
        for token_id_str, token_bytes in model_data["vocab"].items():
            token_id = int(token_id_str)
            self.vocab[token_id] = bytes(token_bytes)
        
        # Load merges
        for pair_str, idx in model_data["merges"].items():
            t1, t2 = map(int, pair_str.split(","))
            self.merges[(t1, t2)] = idx
        
        # Load special tokens
        self.special_tokens = model_data.get("special_tokens", {})
    
    def token_to_str(self, token_id: int) -> str:
        """Get a string representation of a token for visualization."""
        token_bytes = self.vocab[token_id]
        # Try to convert to UTF-8 string if possible
        try:
            s = token_bytes.decode('utf-8')
            # Replace newlines, tabs, etc. for display
            s = s.replace('\n', '\\n').replace('\t', '\\t')
            if len(s.strip()) == 0:
                # If it's all whitespace, show hex
                return f"[hex: {token_bytes.hex()}]"
            return s
        except UnicodeDecodeError:
            # If not a valid UTF-8 string, show hex
            return f"[hex: {token_bytes.hex()}]"
    
    def print_vocab(self, n=50) -> None:
        """Print the first n tokens in the vocabulary for inspection."""
        ids = sorted(self.vocab.keys())
        skipped = max(0, len(ids) - n)
        print(f"Vocabulary size: {len(ids)} tokens")
        print(f"Showing first {min(n, len(ids))} tokens:")
        for i, token_id in enumerate(ids[:n]):
            s = self.token_to_str(token_id)
            print(f"Token {token_id}: {s}")
        if skipped > 0:
            print(f"... and {skipped} more tokens")

## Testing the Tokenizer

Let's test our implementation with a simple example text.

In [3]:
# Create a simple training corpus
training_text = """
Byte Pair Encoding (BPE) is a data compression technique that iteratively replaces the most frequent pair of consecutive bytes in a sequence with a single, unused byte. In NLP, it is used as a subword tokenization algorithm.

The BPE algorithm works as follows:
1. Initialize the vocabulary with individual characters/bytes
2. Count all pairs of adjacent symbols in the training corpus
3. Merge the most frequent pair and add it to the vocabulary
4. Repeat steps 2-3 until reaching the desired vocabulary size

BPE can handle out-of-vocabulary words by splitting them into known subword units, making it effective for various languages and even emoji 👍🌍.
"""

# Initialize our tokenizer
tokenizer = Tokenizer()

# Train to a vocabulary size of 500
tokenizer.train(training_text, vocab_size=500, verbose=True)

# Show some of the learned tokens
tokenizer.print_vocab(30)

Training BPE tokenizer to vocab size 500
Text size: 656 chars, 662 bytes
Merge #0: pair (101, 32) -> 256, corpus now 647 tokens
Merge #100: pair (355, 110) -> 356, corpus now 311 tokens
Merge #200: pair (455, 277) -> 456, corpus now 211 tokens
Vocabulary size: 500 tokens
Showing first 30 tokens:
Token 0:  
Token 1: 
Token 2: 
Token 3: 
Token 4: 
Token 5: 
Token 6: 
Token 7: 
Token 8:
Token 9: \t
Token 10: \n
Token 11: [hex: 0b]
Token 12: [hex: 0c]
Token 13: [hex: 0d]
Token 14: 
Token 15: 
Token 16: 
Token 17: 
Token 18: 
Token 19: 
Token 20: 
Token 21: 
Token 22: 
Token 23: 
Token 24: 
Token 25: 
Token 26: 
Token 27: 
Token 28: [hex: 1c]
Token 29: [hex: 1d]
... and 470 more tokens


## Encoding and Decoding

Now let's test encoding and decoding to verify the tokenizer works as expected.

In [4]:
# Test with a new sentence
test_text = "BPE tokenization works great for natural language processing!"

# Encode the text
encoded = tokenizer.encode(test_text)
print(f"Encoded into {len(encoded)} tokens: {encoded}")

# Display each token
print("\nToken breakdown:")
for i, token_id in enumerate(encoded):
    print(f"Token {i+1}: ID {token_id} = '{tokenizer.token_to_str(token_id)}'")

# Decode the tokens back to text
decoded = tokenizer.decode(encoded)
print(f"\nDecoded text: '{decoded}'")
print(f"Round trip success: {test_text == decoded}")

Encoded into 42 tokens: [292, 32, 305, 107, 269, 306, 97, 266, 111, 293, 282, 107, 257, 103, 265, 311, 102, 262, 32, 110, 97, 116, 117, 295, 108, 32, 267, 110, 103, 117, 97, 103, 256, 112, 114, 111, 99, 101, 115, 115, 264, 33]

Token breakdown:
Token 1: ID 292 = 'BPE'
Token 2: ID 32 = '[hex: 20]'
Token 3: ID 305 = 'to'
Token 4: ID 107 = 'k'
Token 5: ID 269 = 'en'
Token 6: ID 306 = 'iz'
Token 7: ID 97 = 'a'
Token 8: ID 266 = 'ti'
Token 9: ID 111 = 'o'
Token 10: ID 293 = 'n '
Token 11: ID 282 = 'wor'
Token 12: ID 107 = 'k'
Token 13: ID 257 = 's '
Token 14: ID 103 = 'g'
Token 15: ID 265 = 're'
Token 16: ID 311 = 'at '
Token 17: ID 102 = 'f'
Token 18: ID 262 = 'or'
Token 19: ID 32 = '[hex: 20]'
Token 20: ID 110 = 'n'
Token 21: ID 97 = 'a'
Token 22: ID 116 = 't'
Token 23: ID 117 = 'u'
Token 24: ID 295 = 'ra'
Token 25: ID 108 = 'l'
Token 26: ID 32 = '[hex: 20]'
Token 27: ID 267 = 'la'
Token 28: ID 110 = 'n'
Token 29: ID 103 = 'g'
Token 30: ID 117 = 'u'
Token 31: ID 97 = 'a'
Token 32: ID 103 

## Measuring Tokenization Efficiency

Let's compute some metrics on the tokenization efficiency.

In [5]:
def measure_efficiency(tokenizer, texts):
    """Measure tokenization efficiency across multiple text samples."""
    results = []
    for name, text in texts.items():
        # Tokenize and measure
        start_time = time.time()
        tokens = tokenizer.encode(text)
        encode_time = time.time() - start_time
        
        start_time = time.time()
        decoded = tokenizer.decode(tokens)
        decode_time = time.time() - start_time
        
        # Calculate metrics
        char_count = len(text)
        token_count = len(tokens)
        compression_ratio = char_count / token_count
        chars_per_second = char_count / encode_time if encode_time > 0 else 0
        
        # Store results
        results.append({
            "name": name,
            "chars": char_count,
            "tokens": token_count,
            "ratio": compression_ratio,
            "encode_time": encode_time,
            "decode_time": decode_time,
            "chars_per_second": chars_per_second,
            "roundtrip_success": text == decoded
        })
    
    # Print results table
    print(f"{'Text':<15} | {'Chars':<8} | {'Tokens':<8} | {'Ratio':<7} | {'Encode (s)':<10} | {'Decode (s)':<10} | {'Success':<7}")
    print("-" * 75)
    for r in results:
        print(f"{r['name']:<15} | {r['chars']:<8} | {r['tokens']:<8} | {r['ratio']:<7.2f} | {r['encode_time']:<10.4f} | {r['decode_time']:<10.4f} | {r['roundtrip_success']}")
    
    return results

# Define test texts
test_texts = {
    "English": "The quick brown fox jumps over the lazy dog.",
    "Repeated": "hello hello hello hello hello hello hello",
    "Numbers": "1234567890 1234567890 1234567890",
    "Technical": "def factorial(n): return 1 if n <= 1 else n * factorial(n-1)",
    "Emoji": "🙂 🌍 🚀 👨‍👩‍👧‍👦 🎉",
    "Mixed": "Training at 3.5x speed: 😊 快速训练！速度提高"
}

# Measure tokenization efficiency
efficiency_results = measure_efficiency(tokenizer, test_texts)

Text            | Chars    | Tokens   | Ratio   | Encode (s) | Decode (s) | Success
---------------------------------------------------------------------------
English         | 44       | 35       | 1.26    | 0.0001     | 0.0000     | True
Repeated        | 41       | 41       | 1.00    | 0.0000     | 0.0000     | True
Numbers         | 32       | 32       | 1.00    | 0.0000     | 0.0000     | True
Technical       | 60       | 50       | 1.20    | 0.0000     | 0.0000     | True
Emoji           | 15       | 37       | 0.41    | 0.0000     | 0.0000     | True
Mixed           | 35       | 48       | 0.73    | 0.0001     | 0.0000     | True


## Visualizing Tokenization Process

Let's create a visualization of how text gets split into tokens.

In [6]:
def visualize_tokenization(tokenizer, text):
    """Visualize how text is tokenized by showing token boundaries."""
    # Encode the text
    ids = tokenizer.encode(text)
    
    # Get the bytes for each token
    token_bytes = [tokenizer.vocab[id] for id in ids]
    
    # Try to display each token as text
    visualized = []
    for token in token_bytes:
        try:
            token_text = token.decode('utf-8')
            # Replace whitespace for visibility
            token_text = token_text.replace(' ', '␣').replace('\n', '\\n').replace('\t', '\\t')
            visualized.append(token_text)
        except UnicodeDecodeError:
            # If not a valid UTF-8 sequence, show hex
            visualized.append(f"[{token.hex()}]")
    
    # Display with token boundaries
    print(f"Tokenized into {len(ids)} tokens:")
    result = ""
    for token in visualized:
        result += f"[{token}]"
    print(result)
    
    # Display each token with its ID
    print("\nDetailed token breakdown:")
    for i, (id, vis) in enumerate(zip(ids, visualized)):
        print(f"Token {i+1}: ID {id} = '{vis}'")
    
    return visualized

# Visualize tokenization for a sample text
sample_text = "Hello, world! This is a test of BPE tokenization."
tokens_visualized = visualize_tokenization(tokenizer, sample_text)

Tokenized into 34 tokens:
[H][e][l][l][o][,␣][wor][l][d][!][␣][T][h][i][s␣][i][s␣a][␣][te][s][t␣][of][␣][BPE][␣][to][k][en][iz][a][ti][o][n][.]

Detailed token breakdown:
Token 1: ID 72 = 'H'
Token 2: ID 101 = 'e'
Token 3: ID 108 = 'l'
Token 4: ID 108 = 'l'
Token 5: ID 111 = 'o'
Token 6: ID 303 = ',␣'
Token 7: ID 282 = 'wor'
Token 8: ID 108 = 'l'
Token 9: ID 100 = 'd'
Token 10: ID 33 = '!'
Token 11: ID 32 = '␣'
Token 12: ID 84 = 'T'
Token 13: ID 104 = 'h'
Token 14: ID 105 = 'i'
Token 15: ID 257 = 's␣'
Token 16: ID 105 = 'i'
Token 17: ID 277 = 's␣a'
Token 18: ID 32 = '␣'
Token 19: ID 263 = 'te'
Token 20: ID 115 = 's'
Token 21: ID 260 = 't␣'
Token 22: ID 300 = 'of'
Token 23: ID 32 = '␣'
Token 24: ID 292 = 'BPE'
Token 25: ID 32 = '␣'
Token 26: ID 305 = 'to'
Token 27: ID 107 = 'k'
Token 28: ID 269 = 'en'
Token 29: ID 306 = 'iz'
Token 30: ID 97 = 'a'
Token 31: ID 266 = 'ti'
Token 32: ID 111 = 'o'
Token 33: ID 110 = 'n'
Token 34: ID 46 = '.'


## Saving and Loading the Tokenizer

Let's test the serialization and deserialization of our tokenizer.

In [7]:
# Save the tokenizer
tokenizer.save("bpe_tokenizer.json")
print(f"Saved tokenizer with {tokenizer.vocab_size} tokens")

# Create a new tokenizer and load the saved model
new_tokenizer = Tokenizer()
new_tokenizer.load("bpe_tokenizer.json")
print(f"Loaded tokenizer with {new_tokenizer.vocab_size} tokens")

# Verify the loaded tokenizer works the same
check_text = "Testing if the loaded tokenizer works correctly."
original_tokens = tokenizer.encode(check_text)
loaded_tokens = new_tokenizer.encode(check_text)

print(f"Original tokenizer: {len(original_tokens)} tokens")
print(f"Loaded tokenizer: {len(loaded_tokens)} tokens")
print(f"Tokens match: {original_tokens == loaded_tokens}")

Saved tokenizer with 500 tokens
Loaded tokenizer with 500 tokens
Original tokenizer: 32 tokens
Loaded tokenizer: 32 tokens
Tokens match: True


## Regex-Based Tokenizer

For more efficient tokenization in natural language processing, we can implement a regex-based pre-tokenization step before applying BPE merges. This helps the tokenizer better handle natural language boundaries like words and numbers.

In [8]:
class RegexTokenizer(Tokenizer):
    """Enhanced tokenizer with regex-based pre-tokenization."""
    
    def __init__(self):
        super().__init__()
        self.pat = re.compile(r'(\s+|[a-zA-Z]+|[0-9]+|\S)')
        # Ensure all parent class attributes are present
        self.merges = {}  # (token1, token2) -> new_token_id 
        self.vocab = {}   # token_id -> token (bytes)
        self.token_to_id = {}  # token (bytes) -> token_id
        self.vocab_size = 256
        
        # Pre-populate the vocabulary with the basic 256 byte tokens
        for i in range(256):
            token = bytes([i])
            self.vocab[i] = token
            self.token_to_id[token] = i
    
    def encode(self, text):
        """Override encode to use regex-based pre-tokenization."""
        # First split using regex pattern
        parts = [part.encode('utf-8') for part in re.findall(self.pat, text)]
        
        # Then encode each part with the base tokenizer
        ids = []
        for part in parts:
            # Convert to bytes and start with raw byte tokens
            bytes_list = list(part)
            tokens = [bytes([b]) for b in bytes_list]
            
            # Apply merges iteratively, as in the base class
            while len(tokens) >= 2:
                # Find valid merge pairs in the current sequence
                pairs = [(tokens[i], tokens[i+1]) for i in range(len(tokens)-1)]
                valid_pairs = [(pair, self.merges[pair]) for pair in pairs if pair in self.merges]
                
                # If no valid pairs, we're done
                if not valid_pairs:
                    break
                    
                # Find the pair with the lowest merge ID (first learned)
                pair, new_id = min(valid_pairs, key=lambda x: x[1])
                
                # Apply the merge
                tokens = self.merge(tokens, pair, new_id)
            
            # Map tokens to IDs
            ids.extend([self.token_to_id[token] for token in tokens])
            
        return ids

## Special Tokens Support

Let's enhance our tokenizer to support special tokens, similar to GPT models.

In [ ]:
# Define the official GPT text split patterns
GPT2_SPLIT_PATTERN = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
GPT4_SPLIT_PATTERN = r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+"""

# Explanation of GPT4 pattern:
# '(?i:[sdmt]|ll|ve|re) - Matches contractions like 's, 'd, 'm, 't, 'll, 've, 're (case insensitive)
# [^\r\n\p{L}\p{N}]?+\p{L}+ - Matches letter sequences, possibly with a leading non-letter/non-number
# \p{N}{1,3} - Matches 1-3 digit numbers
# ?[^\s\p{L}\p{N}]++[\r\n]* - Matches punctuation and symbols
# \s*[\r\n] - Matches newlines with optional whitespace
# \s+(?!\S) - Matches trailing whitespace
# \s+ - Matches other whitespace sequences

In [ ]:
class RegexTokenizer(Tokenizer):
    """Enhanced tokenizer with regex-based pre-tokenization."""
    
    def __init__(self, pattern=None):
        """
        Initialize the tokenizer with optional regex pattern.
        
        Args:
            pattern: A regex pattern to split text before tokenization.
                    If None, defaults to GPT4_SPLIT_PATTERN.
        """
        super().__init__()
        self.pattern = GPT4_SPLIT_PATTERN if pattern is None else pattern
        self.compiled_pattern = re.compile(self.pattern)
        
        # Ensure all parent class attributes are present
        self.merges = {}  # (token1, token2) -> new_token_id 
        self.vocab = {}   # token_id -> token (bytes)
        self.token_to_id = {}  # token (bytes) -> token_id
        self.vocab_size = 256
        
        # Pre-populate the vocabulary with the basic 256 byte tokens
        for i in range(256):
            token = bytes([i])
            self.vocab[i] = token
            self.token_to_id[token] = i
    
    def train(self, text: str, vocab_size: int, verbose: bool = False) -> None:
        """
        Train the tokenizer with regex-based splitting.
        
        Args:
            text: The training text
            vocab_size: Target vocabulary size
            verbose: Whether to print debug information
        """
        assert vocab_size >= 256
        num_merges = vocab_size - 256
        
        # Split the text into chunks using the regex pattern
        text_chunks = re.findall(self.compiled_pattern, text)
        
        # Process each chunk separately
        ids = [list(ch.encode("utf-8")) for ch in text_chunks]
        
        # Iteratively merge the most common pairs to create new tokens
        merges = {}  # (int, int) -> int
        vocab = {idx: bytes([idx]) for idx in range(256)}  # idx -> bytes
        
        for i in range(num_merges):
            # Count frequencies of adjacent pairs across all chunks
            stats = {}
            for chunk_ids in ids:
                get_stats(chunk_ids, stats)
                
            # If no more pairs, stop early
            if not stats:
                if verbose:
                    print(f"No more pairs to merge after {i} merges")
                break
                
            # Find the most frequent pair
            pair = max(stats, key=stats.get)
            
            # Create a new token for this pair
            idx = 256 + i
            
            # Replace all occurrences of pair with new token
            ids = [merge(chunk_ids, pair, idx) for chunk_ids in ids]
            
            # Record the merge
            merges[pair] = idx
            vocab[idx] = vocab[pair[0]] + vocab[pair[1]]
            
            # Print progress
            if verbose and i % 100 == 0:
                print(f"Merge #{i}: pair {pair} -> {idx}, corpus now has {sum(len(chunk) for chunk in ids)} tokens")
        
        # Save class variables
        self.merges = merges
        self.vocab = vocab
        self.vocab_size = 256 + len(merges)
        
        # Update token_to_id dictionary for efficient encoding
        self.token_to_id = {token: idx for idx, token in vocab.items()}
    
    def encode(self, text: str) -> List[int]:
        """
        Encode text using regex-based preprocessing.
        
        Args:
            text: The text to encode
            
        Returns:
            List of token IDs
        """
        # Split text into chunks using the regex pattern
        text_chunks = re.findall(self.compiled_pattern, text)
        
        # Process each chunk separately and concatenate results
        ids = []
        for chunk in text_chunks:
            chunk_bytes = chunk.encode("utf-8")
            chunk_ids = self._encode_chunk(chunk_bytes)
            ids.extend(chunk_ids)
        
        return ids
    
    def _encode_chunk(self, text_bytes: bytes) -> List[int]:
        """
        Encode a single chunk of bytes.
        
        Args:
            text_bytes: Bytes to encode
            
        Returns:
            List of token IDs
        """
        # Start with raw bytes
        ids = list(text_bytes)
        
        # Apply merges iteratively
        while len(ids) >= 2:
            # Find pairs in the current sequence
            stats = get_stats(ids)
            
            # Find the pair with the lowest merge index
            pair = min(stats, key=lambda p: self.merges.get(p, float("inf")))
            
            # If no more merges possible, stop
            if pair not in self.merges:
                break
                
            # Apply the merge
            idx = self.merges[pair]
            ids = merge(ids, pair, idx)
        
        return ids

In [10]:
# Test special tokens support
# Create common GPT-style special tokens
special_tokens = {
    '<|endoftext|>': 100257,
    '<|fim_prefix|>': 100258,
    '<|fim_middle|>': 100259,
    '<|fim_suffix|>': 100260,
    '<|endofprompt|>': 100261
}

# Create and train a tokenizer with special tokens
special_tokenizer = SpecialTokensTokenizer()
special_tokenizer.train(training_text, vocab_size=500, verbose=False)
special_tokenizer.add_special_tokens(special_tokens)

# Test encoding/decoding with special tokens
test_with_special = "Here is some text <|endoftext|> followed by a special token."
encoded_special = special_tokenizer.encode(test_with_special, allowed_special="all")
decoded_special = special_tokenizer.decode(encoded_special)

print(f"Original text: '{test_with_special}'")
print(f"Encoded with {len(encoded_special)} tokens: {encoded_special}")
print(f"Decoded: '{decoded_special}'")
print(f"Round trip success: {test_with_special == decoded_special}")

# Visualize the tokens including the special token
print("\nToken visualization:")
token_strs = []
for token_id in encoded_special:
    if token_id in special_tokenizer.special_tokens_inv:
        # This is a special token
        token_strs.append(special_tokenizer.special_tokens_inv[token_id])
    else:
        # Regular token
        token_bytes = special_tokenizer.vocab[token_id]
        try:
            token_str = token_bytes.decode('utf-8')
            token_strs.append(token_str)
        except UnicodeDecodeError:
            token_strs.append(f"[hex: {token_bytes.hex()}]")

# Print with token boundaries
result = ""
for token in token_strs:
    if token in special_tokens:
        # Highlight special tokens
        result += f"[*{token}*]"
    else:
        result += f"[{token}]"
print(result)

Original text: 'Here is some text <|endoftext|> followed by a special token.'
Encoded with 33 tokens: [72, 101, 114, 256, 105, 257, 115, 111, 109, 256, 263, 120, 260, 100257, 32, 102, 339, 108, 340, 101, 271, 279, 323, 112, 101, 99, 105, 307, 32, 305, 107, 269, 46]
Decoded: 'Here is some text <|endoftext|> followed by a special token.'
Round trip success: True

Token visualization:
[H][e][r][e ][i][s ][s][o][m][e ][te][x][t ][*<|endoftext|>*][ ][f][ol][l][ow][e][d ][by][ a s][p][e][c][i][al][ ][to][k][en][.]


## Benchmarking Against Reference Tokenizers

Let's compare our tokenizer implementation with other popular tokenizers.

In [ ]:
# Test different regex patterns for tokenization
# Create a training text sample
sample_text = """
Hello world! This is a test of the regex-based tokenization.
It handles contractions like don't, can't, and it's properly.
Numbers like 123 and 45678 are split differently.
Emojis like 😊 and symbols like @ # $ % are also handled.
"""

# Create tokenizers with different patterns
basic_pattern = r'(\s+|[a-zA-Z]+|[0-9]+|\S)'
tokenizer_basic = RegexTokenizer(pattern=basic_pattern)
tokenizer_gpt2 = RegexTokenizer(pattern=GPT2_SPLIT_PATTERN)
tokenizer_gpt4 = RegexTokenizer(pattern=GPT4_SPLIT_PATTERN)

# Train all tokenizers on the same text with the same vocabulary size
vocab_size = 400
for tokenizer, name in zip([tokenizer_basic, tokenizer_gpt2, tokenizer_gpt4], 
                          ["Basic Pattern", "GPT-2 Pattern", "GPT-4 Pattern"]):
    print(f"\nTraining with {name}...")
    tokenizer.train(sample_text, vocab_size, verbose=True)

# Test tokenization with different patterns
test_text = "Hello world! Let's test contractions like can't and numbers like 123 and 4567."

print("\n\nTokenization Comparison:")
for tokenizer, name in zip([tokenizer_basic, tokenizer_gpt2, tokenizer_gpt4], 
                          ["Basic Pattern", "GPT-2 Pattern", "GPT-4 Pattern"]):
    tokens = tokenizer.encode(test_text)
    print(f"\n{name} - {len(tokens)} tokens:")
    
    # Visualize token boundaries
    parts = []
    for token_id in tokens:
        token_bytes = tokenizer.vocab[token_id]
        try:
            token_str = token_bytes.decode('utf-8', errors='replace')
            parts.append(f"[{token_str}]")
        except UnicodeDecodeError:
            parts.append(f"[{token_bytes.hex()}]")
    
    print("".join(parts))

In [ ]:
class SpecialTokensTokenizer(RegexTokenizer):
    """Enhanced tokenizer with support for special tokens and regex pre-tokenization."""
    
    def __init__(self, pattern=None, special_tokens=None):
        """
        Initialize the tokenizer with optional regex pattern and special tokens.
        
        Args:
            pattern: A regex pattern to split text before tokenization.
                    If None, defaults to GPT4_SPLIT_PATTERN.
            special_tokens: Dictionary mapping special token strings to their IDs.
                           Example: {'<|endoftext|>': 100257}
        """
        super().__init__(pattern=pattern)
        
        # Initialize special tokens
        self.special_tokens = {}  # str -> id
        self.inverse_special_tokens = {}  # id -> str
        
        # Register special tokens if provided
        if special_tokens:
            self.register_special_tokens(special_tokens)
    
    def register_special_tokens(self, special_tokens):
        """
        Register special tokens with the tokenizer.
        
        Args:
            special_tokens: Dictionary mapping special token strings to their IDs.
                           Example: {'<|endoftext|>': 100257}
        """
        # Add special tokens to our dictionaries
        for token, idx in special_tokens.items():
            # Make sure ID doesn't conflict with existing vocab
            if idx in self.vocab and idx not in self.inverse_special_tokens:
                raise ValueError(f"ID {idx} already exists in vocabulary")
            
            # Add to special tokens dictionaries
            self.special_tokens[token] = idx
            self.inverse_special_tokens[idx] = token
            
            # Add to vocabulary for decoding
            self.vocab[idx] = token.encode('utf-8')
    
    def encode(self, text, allowed_special="none_raise"):
        """
        Encode text with special token handling.
        
        Args:
            text: The text to encode
            allowed_special: How to handle special tokens.
                            - "all": Process all special tokens
                            - "none": Ignore special tokens (treat as normal text)
                            - "none_raise": Raise error if special tokens present
                            - set(...): Process only specified special tokens
        
        Returns:
            List of token IDs
        """
        # Decode the user desire w.r.t. handling of special tokens
        special = None
        if allowed_special == "all":
            special = self.special_tokens
        elif allowed_special == "none":
            special = {}
        elif allowed_special == "none_raise":
            special = {}
            for token in self.special_tokens:
                if token in text:
                    raise ValueError(f"Special token {token} found in text but not allowed.")
        elif isinstance(allowed_special, set):
            special = {k: v for k, v in self.special_tokens.items() if k in allowed_special}
        else:
            raise ValueError(f"allowed_special={allowed_special} not understood")
        
        # If no special tokens, use the base encoding
        if not special:
            return super().encode(text)
        
        # Handle special tokens by splitting the text
        special_pattern = "(" + "|".join(re.escape(k) for k in special) + ")"
        parts = re.split(special_pattern, text)
        
        # Process each part
        ids = []
        for part in parts:
            if part in special:
                # This is a special token, add its ID
                ids.append(special[part])
            elif part:  # Skip empty strings from split
                # This is regular text, encode it normally
                ids.extend(super().encode(part))
        
        return ids
    
    def decode(self, ids):
        """
        Decode token IDs back to text.
        
        Args:
            ids: List of token IDs
            
        Returns:
            Decoded text
        """
        parts = []
        for idx in ids:
            if idx in self.inverse_special_tokens:
                # This is a special token, decode directly
                parts.append(self.inverse_special_tokens[idx])
            elif idx in self.vocab:
                # This is a regular token, decode from bytes
                parts.append(self.vocab[idx])
            else:
                raise ValueError(f"Invalid token ID: {idx}")
        
        # Join all parts and decode bytes
        text_bytes = b"".join(p.encode('utf-8') if isinstance(p, str) else p for p in parts)
        return text_bytes.decode('utf-8', errors='replace')

In [ ]:
# Test improved special token handling
# Define common GPT special tokens
gpt_special_tokens = {
    '<|endoftext|>': 100257,
    '<|fim_prefix|>': 100258,
    '<|fim_middle|>': 100259,
    '<|fim_suffix|>': 100260,
    '<|endofprompt|>': 100261
}

# Create a special tokens tokenizer
special_tokenizer = SpecialTokensTokenizer(pattern=GPT4_SPLIT_PATTERN, special_tokens=gpt_special_tokens)

# Train it on some text
special_tokenizer.train(sample_text, vocab_size=400, verbose=False)

# Test encoding with special tokens
test_strings = [
    "Hello world",
    "<|endoftext|>Hello world",
    "Hello<|endoftext|>world",
    "Hello world<|endoftext|>",
    "Hello <|fim_prefix|>world<|fim_middle|>test<|fim_suffix|>"
]

print("Special Token Handling Tests:\n")

# Test with different allowed_special settings
for allowed in ["all", "none", set(['<|endoftext|>'])]:
    print(f"\nallowed_special = {allowed}")
    print("-" * 40)
    
    for text in test_strings:
        try:
            tokens = special_tokenizer.encode(text, allowed_special=allowed)
            
            # Show which tokens are special
            token_strs = []
            for token_id in tokens:
                if token_id in special_tokenizer.inverse_special_tokens:
                    # This is a special token
                    special_str = special_tokenizer.inverse_special_tokens[token_id]
                    token_strs.append(f"[*{special_str}*]")
                else:
                    # Regular token
                    token_bytes = special_tokenizer.vocab[token_id]
                    try:
                        token_str = token_bytes.decode('utf-8')
                        token_strs.append(f"[{token_str}]")
                    except UnicodeDecodeError:
                        token_strs.append(f"[hex: {token_bytes.hex()}]")
            
            # Print the result
            print(f"Text: {text}")
            print(f"Tokens: {tokens}")
            print(f"Visualization: {''.join(token_strs)}")
            
            # Test roundtrip
            decoded = special_tokenizer.decode(tokens)
            print(f"Decoded: {decoded}")
            print(f"Roundtrip success: {text == decoded}\n")
            
        except ValueError as e:
            print(f"Text: {text}")
            print(f"Error: {str(e)}\n")

# Test error handling with none_raise
print("\nallowed_special = none_raise (should raise errors)")
print("-" * 40)
for text in test_strings:
    try:
        tokens = special_tokenizer.encode(text, allowed_special="none_raise")
        print(f"Text: {text}")
        print(f"Tokens: {tokens}\n")
    except ValueError as e:
        print(f"Text: {text}")
        print(f"Error: {str(e)}\n")

In [ ]:
# Add Karpathy-compatible save and load functions to base Tokenizer class
def tokenizer_save(self, file_prefix):
    """
    Save the tokenizer in Karpathy's format.
    Creates two files:
    - file_prefix.model: Contains the pattern, special tokens, and merges (used for loading)
    - file_prefix.vocab: Human-readable vocabulary (for inspection only)
    
    Args:
        file_prefix: Path prefix for the saved files
    """
    # Write the model file - used for loading later
    model_file = file_prefix + ".model"
    with open(model_file, 'w') as f:
        # Write version and pattern
        f.write("minbpe v1\n")
        f.write(f"{getattr(self, 'pattern', '')}\n")
        
        # Write special tokens
        special_tokens = getattr(self, 'special_tokens', {})
        f.write(f"{len(special_tokens)}\n")
        for special, idx in special_tokens.items():
            f.write(f"{special} {idx}\n")
            
        # Write the merges
        for (idx1, idx2), idx in self.merges.items():
            f.write(f"{idx1} {idx2}\n")
    
    # Write the vocab file - for human inspection
    vocab_file = file_prefix + ".vocab"
    with open(vocab_file, "w", encoding="utf-8") as f:
        # Build an inverted merges dictionary for visualization
        inverted_merges = {idx: pair for pair, idx in self.merges.items()}
        
        # Write each token with its source if available
        for idx, token in self.vocab.items():
            # Try to decode the token for display
            try:
                token_str = token.decode('utf-8', errors='replace')
                token_str = token_str.replace('\n', '\\n').replace('\t', '\\t')
                if len(token_str.strip()) == 0:
                    # For whitespace, show hex
                    token_str = f"hex: {token.hex()}"
            except:
                token_str = f"hex: {token.hex()}"
                
            # If this token has children, show the merge
            if idx in inverted_merges:
                idx0, idx1 = inverted_merges[idx]
                
                # Get string representations of the children
                try:
                    s0 = self.vocab[idx0].decode('utf-8', errors='replace')
                    s0 = s0.replace('\n', '\\n').replace('\t', '\\t')
                    if len(s0.strip()) == 0:
                        s0 = f"hex: {self.vocab[idx0].hex()}"
                except:
                    s0 = f"hex: {self.vocab[idx0].hex()}"
                    
                try:
                    s1 = self.vocab[idx1].decode('utf-8', errors='replace')
                    s1 = s1.replace('\n', '\\n').replace('\t', '\\t')
                    if len(s1.strip()) == 0:
                        s1 = f"hex: {self.vocab[idx1].hex()}"
                except:
                    s1 = f"hex: {self.vocab[idx1].hex()}"
                
                # Write the merge information
                f.write(f"[{s0}][{s1}] -> [{token_str}] {idx}\n")
            else:
                # This is a leaf token (raw byte or special token)
                f.write(f"[{token_str}] {idx}\n")
                
    print(f"Saved tokenizer to {file_prefix}.model and {file_prefix}.vocab")

def tokenizer_load(self, model_file):
    """
    Load a tokenizer from a .model file in Karpathy's format.
    
    Args:
        model_file: Path to the .model file
    """
    assert model_file.endswith(".model"), "File must have .model extension"
    
    # Reset current state
    self.merges = {}
    special_tokens = {}
    
    # Read the model file
    with open(model_file, 'r', encoding="utf-8") as f:
        # Read version
        version = f.readline().strip()
        assert version == "minbpe v1", f"Unknown model version: {version}"
        
        # Read pattern if available
        pattern = f.readline().strip()
        if hasattr(self, 'pattern'):
            self.pattern = pattern
            self.compiled_pattern = re.compile(pattern)
        
        # Read special tokens
        num_special = int(f.readline().strip())
        for _ in range(num_special):
            line = f.readline().strip()
            special, special_idx = line.split(' ', 1)
            special_tokens[special] = int(special_idx)
        
        # Read merges
        next_idx = 256
        for line in f:
            idx1, idx2 = map(int, line.split())
            self.merges[(idx1, idx2)] = next_idx
            next_idx += 1
    
    # Rebuild vocab
    self.vocab = {idx: bytes([idx]) for idx in range(256)}
    for (p0, p1), idx in self.merges.items():
        self.vocab[idx] = self.vocab[p0] + self.vocab[p1]
    
    # Add special tokens if supported
    if hasattr(self, 'register_special_tokens'):
        self.register_special_tokens(special_tokens)
    elif hasattr(self, 'special_tokens'):
        self.special_tokens = special_tokens
        self.inverse_special_tokens = {v: k for k, v in special_tokens.items()}
        # Add special tokens to vocabulary
        for token, idx in special_tokens.items():
            self.vocab[idx] = token.encode('utf-8')
    
    print(f"Loaded tokenizer from {model_file} with {len(self.merges)} merges")

# Add the save/load methods to the Tokenizer class
Tokenizer.save = tokenizer_save
Tokenizer.load = tokenizer_load

# Test the save and load functionality
def test_save_load():
    # Create and train a tokenizer
    tokenizer = RegexTokenizer(pattern=GPT4_SPLIT_PATTERN)
    tokenizer.train(sample_text, vocab_size=400, verbose=False)
    
    # Save the tokenizer
    tokenizer.save("test_tokenizer")
    
    # Load a new tokenizer and compare
    new_tokenizer = RegexTokenizer()
    new_tokenizer.load("test_tokenizer.model")
    
    # Test encoding/decoding with both tokenizers
    test_text = "Hello world! This is a tokenizer test."
    original_tokens = tokenizer.encode(test_text)
    loaded_tokens = new_tokenizer.encode(test_text)
    
    print(f"Original tokens: {original_tokens}")
    print(f"Loaded tokens: {loaded_tokens}")
    print(f"Tokens match: {original_tokens == loaded_tokens}")
    
    # Test special tokens tokenizer
    special_tokenizer = SpecialTokensTokenizer(
        pattern=GPT4_SPLIT_PATTERN, 
        special_tokens=gpt_special_tokens
    )
    special_tokenizer.train(sample_text, vocab_size=400, verbose=False)
    
    # Save and load
    special_tokenizer.save("test_special_tokenizer")
    
    new_special_tokenizer = SpecialTokensTokenizer()
    new_special_tokenizer.load("test_special_tokenizer.model")
    
    # Test with special tokens
    special_text = "Hello<|endoftext|>world"
    original_special = special_tokenizer.encode(special_text, allowed_special="all")
    loaded_special = new_special_tokenizer.encode(special_text, allowed_special="all")
    
    print(f"\nSpecial tokens test:")
    print(f"Original tokens: {original_special}")
    print(f"Loaded tokens: {loaded_special}")
    print(f"Tokens match: {original_special == loaded_special}")
    
    # Clean up test files
    import os
    for file in ["test_tokenizer.model", "test_tokenizer.vocab", 
                "test_special_tokenizer.model", "test_special_tokenizer.vocab"]:
        if os.path.exists(file):
            os.remove(file)

# Run the test
test_save_load()

In [12]:
# This cell would normally import and compare with reference tokenizers
# We provide pseudocode here as an example of what you would do

"""
# You would usually import tiktoken or transformers:
# import tiktoken
# from transformers import GPT2Tokenizer

# Define benchmark texts and functions
benchmark_texts = {
    "English": "The quick brown fox jumps over the lazy dog.",
    "Code": "def factorial(n): return 1 if n <= 1 else n * factorial(n-1)",
    "Mixed": "Training at 3.5x speed 😊 быстрое обучение",
    "Repeated": "token token token token token token"
}

def benchmark_tokenizer(name, tokenizer_func, texts):
    results = []
    for text_name, text in texts.items():
        # Measure encoding time
        start_time = time.time()
        tokens = tokenizer_func(text)
        encode_time = time.time() - start_time
        
        results.append({
            "tokenizer": name,
            "text": text_name,
            "tokens": len(tokens),
            "encode_time_ms": encode_time * 1000
        })
    return results

# Define encoding functions for different tokenizers
def encode_with_our_tokenizer(text):
    return tokenizer.encode(text)

# def encode_with_tiktoken(text):
#     enc = tiktoken.get_encoding("gpt2")
#     return enc.encode(text)
    
# def encode_with_hf_tokenizer(text):
#     hf_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
#     return hf_tokenizer.encode(text)

# Run benchmarks
our_results = benchmark_tokenizer("Our BPE", encode_with_our_tokenizer, benchmark_texts)
# tiktoken_results = benchmark_tokenizer("tiktoken", encode_with_tiktoken, benchmark_texts)
# hf_results = benchmark_tokenizer("HuggingFace", encode_with_hf_tokenizer, benchmark_texts)

# all_results = our_results + tiktoken_results + hf_results
all_results = our_results

# Display results in a table
print(f"{'Tokenizer':<15} | {'Text':<10} | {'Tokens':<8} | {'Time (ms)':<10}")
print("-" * 50)
for r in all_results:
    print(f"{r['tokenizer']:<15} | {r['text']:<10} | {r['tokens']:<8} | {r['encode_time_ms']:<10.2f}")

# Here you would normally create visualization comparing the tokenizers
"""

# For now, we'll just run a simple benchmark on our own tokenizer
benchmark_texts = {
    "English": "The quick brown fox jumps over the lazy dog.",
    "Code": "def factorial(n): return 1 if n <= 1 else n * factorial(n-1)",
    "Mixed": "Training at 3.5x speed 😊 快速训练！",
    "Repeated": "token token token token token token"
}

def benchmark_our_tokenizers(texts):
    results = []
    tokenizers = {
        "Basic BPE": tokenizer,
        "Regex BPE": regex_tokenizer,
        "Special BPE": special_tokenizer
    }
    
    for name, tkn in tokenizers.items():
        for text_name, text in texts.items():
            # Measure encoding time
            start_time = time.time()
            
            # For special tokenizer, specify allowed_special
            if name == "Special BPE":
                tokens = tkn.encode(text, allowed_special="all")
            else:
                tokens = tkn.encode(text)
                
            encode_time = time.time() - start_time
            
            results.append({
                "tokenizer": name,
                "text": text_name,
                "tokens": len(tokens),
                "encode_time_ms": encode_time * 1000
            })
    return results

results = benchmark_our_tokenizers(benchmark_texts)

# Display results in a table
print(f"{'Tokenizer':<15} | {'Text':<10} | {'Tokens':<8} | {'Time (ms)':<10}")
print("-" * 50)
for r in results:
    print(f"{r['tokenizer']:<15} | {r['text']:<10} | {r['tokens']:<8} | {r['encode_time_ms']:<10.2f}")

Tokenizer       | Text       | Tokens   | Time (ms) 
--------------------------------------------------
Basic BPE       | English    | 35       | 0.05      
Basic BPE       | Code       | 50       | 0.04      
Basic BPE       | Mixed      | 34       | 0.04      
Basic BPE       | Repeated   | 23       | 0.01      
Regex BPE       | English    | 44       | 0.25      
Regex BPE       | Code       | 60       | 0.07      
Regex BPE       | Mixed      | 43       | 0.04      
Regex BPE       | Repeated   | 35       | 0.02      
Special BPE     | English    | 35       | 0.09      
Special BPE     | Code       | 50       | 0.08      
Special BPE     | Mixed      | 34       | 0.06      
Special BPE     | Repeated   | 23       | 0.03      


In [13]:
# Train the regex tokenizer first
regex_tokenizer = RegexTokenizer()
regex_tokenizer.train(training_text, vocab_size=500, verbose=False)

# Now we can do benchmarking
benchmark_texts = {
    "English": "The quick brown fox jumps over the lazy dog.",
    "Code": "def factorial(n): return 1 if n <= 1 else n * factorial(n-1)",
    "Mixed": "Training at 3.5x speed 😊 快速训练！",
    "Repeated": "token token token token token token"
}

def benchmark_our_tokenizers(texts):
    results = []
    tokenizers = {
        "Basic BPE": tokenizer,
        "Regex BPE": regex_tokenizer,
        "Special BPE": special_tokenizer
    }
    
    for name, tkn in tokenizers.items():
        for text_name, text in texts.items():
            # Measure encoding time
            start_time = time.time()
            
            # For special tokenizer, specify allowed_special
            if name == "Special BPE":
                tokens = tkn.encode(text, allowed_special="all")
            else:
                tokens = tkn.encode(text)
                
            encode_time = time.time() - start_time
            
            results.append({
                "tokenizer": name,
                "text": text_name,
                "tokens": len(tokens),
                "encode_time_ms": encode_time * 1000
            })
    return results

results = benchmark_our_tokenizers(benchmark_texts)

# Display results in a table
print(f"{'Tokenizer':<15} | {'Text':<10} | {'Tokens':<8} | {'Time (ms)':<10}")
print("-" * 50)
for r in results:
    print(f"{r['tokenizer']:<15} | {r['text']:<10} | {r['tokens']:<8} | {r['encode_time_ms']:<10.2f}")

Tokenizer       | Text       | Tokens   | Time (ms) 
--------------------------------------------------
Basic BPE       | English    | 35       | 0.05      
Basic BPE       | Code       | 50       | 0.04      
Basic BPE       | Mixed      | 34       | 0.04      
Basic BPE       | Repeated   | 23       | 0.01      
Regex BPE       | English    | 44       | 0.07      
Regex BPE       | Code       | 60       | 0.15      
Regex BPE       | Mixed      | 43       | 0.05      
Regex BPE       | Repeated   | 35       | 0.02      
Special BPE     | English    | 35       | 0.08      
Special BPE     | Code       | 50       | 0.09      
Special BPE     | Mixed      | 34       | 0.06      
Special BPE     | Repeated   | 23       | 0.03      


## Enhanced Token Visualization

Let's improve our token visualization to better illustrate token boundaries and include additional token information.

## Testing the Regex Tokenizer

Let's compare the basic tokenizer with our regex-based tokenizer to see the differences.

In [14]:
# Compare with the basic tokenizer
compare_text = "It's not just tokenization, it's BPE tokenization with regex pre-processing!"

basic_tokens = tokenizer.encode(compare_text)
regex_tokens = regex_tokenizer.encode(compare_text)

print(f"Basic tokenizer: {len(basic_tokens)} tokens")
print(f"Regex tokenizer: {len(regex_tokens)} tokens")

# Visualize the differences
print("\nBasic tokenization:")
visualize_tokenization(tokenizer, compare_text)

print("\nRegex tokenization:")
visualize_tokenization(regex_tokenizer, compare_text)

Basic tokenizer: 51 tokens
Regex tokenizer: 76 tokens

Basic tokenization:
Tokenized into 51 tokens:
[I][t]['][s␣][n][o][t␣][j][u][s][t␣][to][k][en][iz][a][ti][o][n][,␣][it]['][s␣][BPE][␣][to][k][en][iz][a][ti][o][n␣][with][␣re][g][e][x][␣][p][re][-][p][r][o][c][e][s][s][ing][!]

Detailed token breakdown:
Token 1: ID 73 = 'I'
Token 2: ID 116 = 't'
Token 3: ID 39 = '''
Token 4: ID 257 = 's␣'
Token 5: ID 110 = 'n'
Token 6: ID 111 = 'o'
Token 7: ID 260 = 't␣'
Token 8: ID 106 = 'j'
Token 9: ID 117 = 'u'
Token 10: ID 115 = 's'
Token 11: ID 260 = 't␣'
Token 12: ID 305 = 'to'
Token 13: ID 107 = 'k'
Token 14: ID 269 = 'en'
Token 15: ID 306 = 'iz'
Token 16: ID 97 = 'a'
Token 17: ID 266 = 'ti'
Token 18: ID 111 = 'o'
Token 19: ID 110 = 'n'
Token 20: ID 303 = ',␣'
Token 21: ID 346 = 'it'
Token 22: ID 39 = '''
Token 23: ID 257 = 's␣'
Token 24: ID 292 = 'BPE'
Token 25: ID 32 = '␣'
Token 26: ID 305 = 'to'
Token 27: ID 107 = 'k'
Token 28: ID 269 = 'en'
Token 29: ID 306 = 'iz'
Token 30: ID 97 = 'a'
Tok

['I',
 't',
 "'",
 's',
 '␣',
 'n',
 'o',
 't',
 '␣',
 'j',
 'u',
 's',
 't',
 '␣',
 't',
 'o',
 'k',
 'e',
 'n',
 'i',
 'z',
 'a',
 't',
 'i',
 'o',
 'n',
 ',',
 '␣',
 'i',
 't',
 "'",
 's',
 '␣',
 'B',
 'P',
 'E',
 '␣',
 't',
 'o',
 'k',
 'e',
 'n',
 'i',
 'z',
 'a',
 't',
 'i',
 'o',
 'n',
 '␣',
 'w',
 'i',
 't',
 'h',
 '␣',
 'r',
 'e',
 'g',
 'e',
 'x',
 '␣',
 'p',
 'r',
 'e',
 '-',
 'p',
 'r',
 'o',
 'c',
 'e',
 's',
 's',
 'i',
 'n',
 'g',
 '!']

## Testing the Regex Tokenizer

In [15]:
# Train the regex tokenizer
regex_tokenizer = RegexTokenizer()
regex_tokenizer.train(training_text, vocab_size=500, verbose=True)

# Compare with the basic tokenizer
compare_text = "It's not just tokenization, it's BPE tokenization with regex pre-processing!"

basic_tokens = tokenizer.encode(compare_text)
regex_tokens = regex_tokenizer.encode(compare_text)

print(f"Basic tokenizer: {len(basic_tokens)} tokens")
print(f"Regex tokenizer: {len(regex_tokens)} tokens")

# Visualize the differences
print("\nBasic tokenization:")
visualize_tokenization(tokenizer, compare_text)

print("\nRegex tokenization:")
visualize_tokenization(regex_tokenizer, compare_text)

Training BPE tokenizer to vocab size 500
Text size: 656 chars, 662 bytes
Merge #0: pair (101, 32) -> 256, corpus now 647 tokens
Merge #100: pair (355, 110) -> 356, corpus now 311 tokens
Merge #200: pair (455, 277) -> 456, corpus now 211 tokens
Basic tokenizer: 51 tokens
Regex tokenizer: 76 tokens

Basic tokenization:
Tokenized into 51 tokens:
[I][t]['][s␣][n][o][t␣][j][u][s][t␣][to][k][en][iz][a][ti][o][n][,␣][it]['][s␣][BPE][␣][to][k][en][iz][a][ti][o][n␣][with][␣re][g][e][x][␣][p][re][-][p][r][o][c][e][s][s][ing][!]

Detailed token breakdown:
Token 1: ID 73 = 'I'
Token 2: ID 116 = 't'
Token 3: ID 39 = '''
Token 4: ID 257 = 's␣'
Token 5: ID 110 = 'n'
Token 6: ID 111 = 'o'
Token 7: ID 260 = 't␣'
Token 8: ID 106 = 'j'
Token 9: ID 117 = 'u'
Token 10: ID 115 = 's'
Token 11: ID 260 = 't␣'
Token 12: ID 305 = 'to'
Token 13: ID 107 = 'k'
Token 14: ID 269 = 'en'
Token 15: ID 306 = 'iz'
Token 16: ID 97 = 'a'
Token 17: ID 266 = 'ti'
Token 18: ID 111 = 'o'
Token 19: ID 110 = 'n'
Token 20: ID 303 

['I',
 't',
 "'",
 's',
 '␣',
 'n',
 'o',
 't',
 '␣',
 'j',
 'u',
 's',
 't',
 '␣',
 't',
 'o',
 'k',
 'e',
 'n',
 'i',
 'z',
 'a',
 't',
 'i',
 'o',
 'n',
 ',',
 '␣',
 'i',
 't',
 "'",
 's',
 '␣',
 'B',
 'P',
 'E',
 '␣',
 't',
 'o',
 'k',
 'e',
 'n',
 'i',
 'z',
 'a',
 't',
 'i',
 'o',
 'n',
 '␣',
 'w',
 'i',
 't',
 'h',
 '␣',
 'r',
 'e',
 'g',
 'e',
 'x',
 '␣',
 'p',
 'r',
 'e',
 '-',
 'p',
 'r',
 'o',
 'c',
 'e',
 's',
 's',
 'i',
 'n',
 'g',
 '!']

<cell_type>markdown</cell_type>## GPT4Tokenizer Implementation

Modern tokenizers like those used in OpenAI's models employ several advanced techniques to improve tokenization efficiency and quality:

1. **Pattern-Based Splitting**: Uses sophisticated regex patterns to split text along linguistic boundaries
2. **Byte Shuffling**: Reorganizes byte values to improve compression of non-ASCII characters
3. **Special Token Handling**: Manages model-specific tokens like `<|endoftext|>` with configurable behavior

The GPT4Tokenizer implements all these features to provide compatibility with tokenizers like tiktoken's cl100k_base. This enables our educational implementation to produce results comparable to production tokenizers.

### Byte Shuffling Explanation

Byte shuffling is a technique that permutes the byte values (particularly ASCII visible characters) to improve how the BPE algorithm learns merges for multilingual text. Here's how it works:

1. Define a fixed permutation of byte values (using a seeded RNG for reproducibility)
2. During encoding: shuffle bytes before applying BPE merges
3. During decoding: unshuffle bytes after converting tokens back to bytes

This approach helps the tokenizer handle non-ASCII characters more efficiently by making the distribution of byte n-grams more amenable to compression through BPE merges.

In [ ]:
class GPT4Tokenizer(SpecialTokensTokenizer):
    """
    GPT4-compatible tokenizer with regex-based pre-tokenization and byte shuffling.
    Designed to be compatible with tiktoken's cl100k_base encoding.
    """
    
    def __init__(self, pattern=None, special_tokens=None):
        """
        Initialize the GPT4Tokenizer.
        
        Args:
            pattern: A regex pattern to split text before tokenization.
                    If None, defaults to GPT4_SPLIT_PATTERN.
            special_tokens: Dictionary mapping special token strings to their IDs.
                           Example: {'<|endoftext|>': 100257}
        """
        # Default to GPT4 pattern if none provided
        pattern = GPT4_SPLIT_PATTERN if pattern is None else pattern
        
        # Initialize with default GPT special tokens if none provided
        if special_tokens is None:
            special_tokens = {
                '<|endoftext|>': 100257,
                '<|fim_prefix|>': 100258,
                '<|fim_middle|>': 100259,
                '<|fim_suffix|>': 100260,
                '<|endofprompt|>': 100261
            }
        
        super().__init__(pattern=pattern, special_tokens=special_tokens)
        
        # Setup byte shuffling
        self.byte_shuffle = self._get_byte_shuffle()
        self.byte_unshuffle = {v: k for k, v in self.byte_shuffle.items()}
    
    def _get_byte_shuffle(self):
        """
        Create the byte shuffling mapping used by cl100k_base.
        This shuffling is what allows the tokenizer to encode non-ASCII
        characters more efficiently.
        
        Returns:
            Dictionary mapping original byte values to shuffled values
        """
        # Define the permutation pattern from cl100k_base
        bs = list(range(ord("!"), ord("~") + 1))  # ASCII visible characters
        cs = bs.copy()
        n = len(bs)
        
        # Apply the permutation
        # This is a reproducible scrambling of byte values
        rng = random.Random(42)  # Fixed seed for reproduciblity
        for i in range(n - 1, 0, -1):
            j = rng.randint(0, i)
            cs[i], cs[j] = cs[j], cs[i]
        
        # Create the byte shuffle mapping
        shuffle = {b: c for b, c in zip(bs, cs)}
        
        # Add identity mapping for all other bytes
        for i in range(256):
            if i not in shuffle:
                shuffle[i] = i
                
        return shuffle
    
    def _encode_chunk(self, text_bytes):
        """
        Encode a single chunk of bytes with byte shuffling.
        
        Args:
            text_bytes: Bytes to encode
            
        Returns:
            List of token IDs
        """
        # Apply byte shuffling
        shuffled_bytes = bytes(self.byte_shuffle[b] for b in text_bytes)
        
        # Encode using the base implementation
        return super()._encode_chunk(shuffled_bytes)
    
    def decode(self, ids):
        """
        Decode token IDs back to text with byte unshuffling.
        
        Args:
            ids: List of token IDs
            
        Returns:
            Decoded text
        """
        # Decode using the base implementation
        result = super().decode(ids)
        
        # Apply byte unshuffling
        result_bytes = result.encode('utf-8')
        unshuffled_bytes = bytes(self.byte_unshuffle[b] for b in result_bytes)
        
        # Convert back to text
        return unshuffled_bytes.decode('utf-8', errors='replace')
    
    @classmethod
    def create_from_tiktoken(cls, encoding_name="cl100k_base"):
        """
        Create a GPT4Tokenizer from a tiktoken encoding.
        This allows you to use the same vocabulary as OpenAI's models.
        
        Args:
            encoding_name: The name of the tiktoken encoding to use
            
        Returns:
            A GPT4Tokenizer initialized with the tiktoken vocabulary
            
        Note:
            This method requires the tiktoken package to be installed.
            If not available, it will raise an ImportError.
        """
        try:
            import tiktoken
        except ImportError:
            raise ImportError("The tiktoken package is required for this feature. "
                              "Install it with 'pip install tiktoken'.")
        
        # Create the base tokenizer
        tokenizer = cls()
        
        # Get the tiktoken encoding
        enc = tiktoken.get_encoding(encoding_name)
        
        # This is a simplified approximation - in a full implementation,
        # we would extract the exact merges and special tokens from tiktoken,
        # but that's beyond the scope of this example
        print(f"Created GPT4Tokenizer based on {encoding_name}")
        print("Note: This is a simplified compatibility layer")
        
        return tokenizer

<cell_type>markdown</cell_type>## Testing GPT4Tokenizer

Let's test our GPT4Tokenizer implementation with various text inputs to verify it works correctly, especially with special tokens and byte shuffling.

In [ ]:
# Create and train a GPT4Tokenizer
gpt4_tokenizer = GPT4Tokenizer()

# Train it on some sample text
gpt4_training_text = """
This is sample text for training our GPT4Tokenizer.
It includes some code: def hello(): print("Hello, world!")
As well as emojis 😊 🚀 and special characters: $%^&*()
Let's also test some multilingual text: こんにちは, Привет, مرحبا, 你好
"""

gpt4_tokenizer.train(gpt4_training_text, vocab_size=500, verbose=True)

# Test texts with different characteristics
test_texts = {
    "English": "The quick brown fox jumps over the lazy dog.",
    "Code": "def factorial(n): return 1 if n <= 1 else n * factorial(n-1)",
    "Emojis": "I love coding! 😊 💻 🚀 🔥",
    "Special Tokens": "<|endoftext|>Hello world<|fim_prefix|>",
    "Multilingual": "Hello in Japanese is こんにちは and in Russian is Привет",
    "Mixed": "Training at 3.5x speed 😊 快速训练！"
}

# Test encoding and decoding
print("GPT4Tokenizer Testing:")
print("=" * 50)

for name, text in test_texts.items():
    # For special tokens test, use allowed_special="all"
    allowed_special = "all" if name == "Special Tokens" else "none"
    
    # Encode the text
    try:
        tokens = gpt4_tokenizer.encode(text, allowed_special=allowed_special)
        
        # Decode back to text
        decoded = gpt4_tokenizer.decode(tokens)
        
        # Check if roundtrip was successful
        success = text == decoded
        
        print(f"\nText type: {name}")
        print(f"Original: '{text}'")
        print(f"Encoded: {tokens[:10]}{'...' if len(tokens) > 10 else ''} ({len(tokens)} tokens)")
        print(f"Decoded: '{decoded}'")
        print(f"Roundtrip success: {success}")
        
        # If roundtrip failed, show where the difference is
        if not success:
            print("Difference:")
            for i, (a, b) in enumerate(zip(text, decoded)):
                if a != b:
                    print(f"Position {i}: '{a}' vs '{b}'")
                    break
    
    except Exception as e:
        print(f"\nText type: {name}")
        print(f"Error: {str(e)}")

# Test byte shuffling specifically
print("\nByte Shuffling Test:")
print("=" * 50)

# Create text with bytes that would be shuffled
text_with_special_bytes = "!@#$%^&*()_+{}|:\"<>?~`-=[]\\;',./"
encoded_shuffled = gpt4_tokenizer.encode(text_with_special_bytes)
decoded_shuffled = gpt4_tokenizer.decode(encoded_shuffled)

print(f"Original: '{text_with_special_bytes}'")
print(f"Encoded: {encoded_shuffled}")
print(f"Decoded: '{decoded_shuffled}'")
print(f"Roundtrip success: {text_with_special_bytes == decoded_shuffled}")

# Visualization of token boundaries with special tokens
print("\nToken Boundary Visualization:")
print("=" * 50)

mixed_text = "Hello<|endoftext|>World! Let's see 😊 how this works."
encoded_mixed = gpt4_tokenizer.encode(mixed_text, allowed_special="all")

# Create a visualization with token boundaries
tokens_vis = []
for token_id in encoded_mixed:
    if token_id in gpt4_tokenizer.inverse_special_tokens:
        # This is a special token
        special_str = gpt4_tokenizer.inverse_special_tokens[token_id]
        tokens_vis.append(f"[*{special_str}*]")
    else:
        # Regular token
        token_bytes = gpt4_tokenizer.vocab[token_id]
        try:
            token_str = token_bytes.decode('utf-8')
            tokens_vis.append(f"[{token_str}]")
        except UnicodeDecodeError:
            tokens_vis.append(f"[hex: {token_bytes.hex()}]")

print(f"Text: '{mixed_text}'")
print(f"Tokenized: {''.join(tokens_vis)}")

<cell_type>markdown</cell_type>## Optional: tiktoken Compatibility

The following demonstrates how we could extend our tokenizer to be compatible with tiktoken's encodings. This is optional and requires the tiktoken package to be installed.

In [ ]:
# This cell is commented out as it requires tiktoken to be installed
# Uncomment and run if you have tiktoken available

"""
try:
    import tiktoken
    
    # Compare our tokenizer with tiktoken
    def compare_with_tiktoken(text, encoding_name="cl100k_base"):
        # Get tiktoken encoding
        enc = tiktoken.get_encoding(encoding_name)
        tiktoken_tokens = enc.encode(text)
        
        # Use our GPT4Tokenizer
        # In a full implementation, we would initialize our tokenizer
        # with the exact same merges as tiktoken, but for this example
        # we'll just show the concept
        our_tokens = gpt4_tokenizer.encode(text, allowed_special="all")
        
        print(f"Text: '{text}'")
        print(f"tiktoken ({encoding_name}): {tiktoken_tokens}")
        print(f"Our GPT4Tokenizer: {our_tokens}")
        
        # For a basic comparison, we can check if the token count is similar
        print(f"Token count comparison: tiktoken={len(tiktoken_tokens)}, ours={len(our_tokens)}")
        
        return tiktoken_tokens, our_tokens
    
    # Try with different texts
    test_samples = [
        "Hello world",
        "The quick brown fox jumps over the lazy dog",
        "def hello(): print('Hello, world!')",
        "😊 🚀 💻",
        "<|endoftext|>Special token test"
    ]
    
    for sample in test_samples:
        print("\n" + "="*50)
        tiktoken_tokens, our_tokens = compare_with_tiktoken(sample)

except ImportError:
    print("tiktoken is not installed. Install with: pip install tiktoken")
"""

<cell_type>markdown</cell_type>## Karpathy-Compatible Save/Load Functionality

Tokenizer persistence is crucial for practical applications. Our implementation follows Karpathy's minbpe format for compatibility:

### File Format

1. **`.model` File**: Contains all information needed to recreate the tokenizer
   - Header: `minbpe v1`
   - Regex pattern (if applicable)
   - Special tokens count and mappings
   - Merge pairs in the order they were learned

2. **`.vocab` File**: Human-readable vocabulary for inspection
   - Shows each token with its string representation
   - For merged tokens, shows the constituent parts
   - Includes formatting for whitespace and non-printable characters

### Benefits of This Approach

- **Compact Representation**: Only stores essential information (merges and special tokens)
- **Human Readability**: The .vocab file allows for inspection and debugging
- **Compatibility**: Works with Karpathy's original implementation
- **Class Flexibility**: Works with any tokenizer class through polymorphism

The implementation handles all tokenizer variants including those with regex patterns, special tokens, and byte shuffling, ensuring consistent behavior after serialization and deserialization.

In [ ]:
def save_tokenizer_karpathy_format(tokenizer, file_prefix):
    """
    Save a tokenizer in Karpathy's format.
    Creates two files:
    - file_prefix.model: Contains the pattern, special tokens, and merges (used for loading)
    - file_prefix.vocab: Human-readable vocabulary (for inspection only)
    
    Args:
        tokenizer: The tokenizer to save
        file_prefix: Path prefix for the saved files
    """
    # Write the model file - used for loading later
    model_file = file_prefix + ".model"
    with open(model_file, 'w', encoding='utf-8') as f:
        # Write version and pattern
        f.write("minbpe v1\n")
        
        # Write pattern if available
        pattern = getattr(tokenizer, 'pattern', '')
        f.write(f"{pattern}\n")
        
        # Write special tokens
        special_tokens = getattr(tokenizer, 'special_tokens', {})
        f.write(f"{len(special_tokens)}\n")
        for special, idx in special_tokens.items():
            f.write(f"{special} {idx}\n")
            
        # Write the merges
        for (idx1, idx2), idx in tokenizer.merges.items():
            f.write(f"{idx1} {idx2}\n")
    
    # Write the vocab file - for human inspection
    vocab_file = file_prefix + ".vocab"
    with open(vocab_file, "w", encoding="utf-8") as f:
        # Build an inverted merges dictionary for visualization
        inverted_merges = {idx: pair for pair, idx in tokenizer.merges.items()}
        
        # Write each token with its source if available
        for idx, token in sorted(tokenizer.vocab.items()):
            # Try to decode the token for display
            try:
                token_str = token.decode('utf-8', errors='replace')
                token_str = token_str.replace('\n', '\\n').replace('\t', '\\t')
                if len(token_str.strip()) == 0:
                    # For whitespace, show hex
                    token_str = f"hex: {token.hex()}"
            except:
                token_str = f"hex: {token.hex()}"
                
            # If this token has children, show the merge
            if idx in inverted_merges:
                idx0, idx1 = inverted_merges[idx]
                
                # Get string representations of the children
                try:
                    s0 = tokenizer.vocab[idx0].decode('utf-8', errors='replace')
                    s0 = s0.replace('\n', '\\n').replace('\t', '\\t')
                    if len(s0.strip()) == 0:
                        s0 = f"hex: {tokenizer.vocab[idx0].hex()}"
                except:
                    s0 = f"hex: {tokenizer.vocab[idx0].hex()}"
                    
                try:
                    s1 = tokenizer.vocab[idx1].decode('utf-8', errors='replace')
                    s1 = s1.replace('\n', '\\n').replace('\t', '\\t')
                    if len(s1.strip()) == 0:
                        s1 = f"hex: {tokenizer.vocab[idx1].hex()}"
                except:
                    s1 = f"hex: {tokenizer.vocab[idx1].hex()}"
                
                # Write the merge information
                f.write(f"[{s0}][{s1}] -> [{token_str}] {idx}\n")
            else:
                # This is a leaf token (raw byte or special token)
                f.write(f"[{token_str}] {idx}\n")
                
    print(f"Saved tokenizer to {model_file} and {vocab_file}")

def load_tokenizer_karpathy_format(tokenizer_class, model_file):
    """
    Load a tokenizer from a .model file in Karpathy's format.
    
    Args:
        tokenizer_class: The tokenizer class to instantiate (e.g., Tokenizer, RegexTokenizer)
        model_file: Path to the .model file
        
    Returns:
        An instance of the tokenizer class loaded with the model
    """
    assert model_file.endswith(".model"), "File must have .model extension"
    
    # Create a new tokenizer instance
    tokenizer = tokenizer_class()
    
    # Read the model file
    with open(model_file, 'r', encoding="utf-8") as f:
        # Read version
        version = f.readline().strip()
        assert version == "minbpe v1", f"Unknown model version: {version}"
        
        # Read pattern if available
        pattern = f.readline().strip()
        if hasattr(tokenizer, 'pattern'):
            tokenizer.pattern = pattern
            tokenizer.compiled_pattern = re.compile(pattern)
        
        # Read special tokens
        num_special = int(f.readline().strip())
        special_tokens = {}
        for _ in range(num_special):
            line = f.readline().strip()
            special, special_idx = line.split(' ', 1)
            special_tokens[special] = int(special_idx)
        
        # Read merges
        tokenizer.merges = {}
        next_idx = 256
        for line in f:
            try:
                idx1, idx2 = map(int, line.strip().split())
                tokenizer.merges[(idx1, idx2)] = next_idx
                next_idx += 1
            except:
                # Skip malformed lines
                continue
    
    # Rebuild vocab
    tokenizer.vocab = {idx: bytes([idx]) for idx in range(256)}
    for (p0, p1), idx in tokenizer.merges.items():
        if p0 in tokenizer.vocab and p1 in tokenizer.vocab:
            tokenizer.vocab[idx] = tokenizer.vocab[p0] + tokenizer.vocab[p1]
    
    # Set vocabulary size
    tokenizer.vocab_size = max(tokenizer.vocab.keys()) + 1 if tokenizer.vocab else 256
    
    # Add special tokens if supported
    if hasattr(tokenizer, 'register_special_tokens'):
        tokenizer.register_special_tokens(special_tokens)
    elif hasattr(tokenizer, 'special_tokens'):
        tokenizer.special_tokens = special_tokens
        tokenizer.inverse_special_tokens = {v: k for k, v in special_tokens.items()}
        # Add special tokens to vocabulary
        for token, idx in special_tokens.items():
            tokenizer.vocab[idx] = token.encode('utf-8')
    
    # Update token_to_id for efficient encoding if applicable
    if hasattr(tokenizer, 'token_to_id'):
        tokenizer.token_to_id = {token: idx for idx, token in tokenizer.vocab.items()}
    
    print(f"Loaded tokenizer from {model_file} with {len(tokenizer.merges)} merges")
    return tokenizer

# Monkey patch the Tokenizer class and its subclasses to add these methods
Tokenizer.save_karpathy_format = save_tokenizer_karpathy_format
Tokenizer.load_karpathy_format = classmethod(lambda cls, model_file: load_tokenizer_karpathy_format(cls, model_file))

<cell_type>markdown</cell_type>## Testing Save/Load Functionality

Let's test our Karpathy-compatible save and load functionality with different tokenizer types to ensure compatibility.

In [ ]:
# Test the Karpathy-compatible save/load functionality across tokenizer types
import os

def test_save_load_karpathy_format(tokenizer, test_text, allowed_special="none"):
    """Test save and load functionality for a tokenizer."""
    
    # Get tokenizer name for the file prefix
    tokenizer_name = tokenizer.__class__.__name__
    file_prefix = f"test_{tokenizer_name.lower()}"
    
    # Save the tokenizer
    tokenizer.save_karpathy_format(file_prefix)
    
    # Load the tokenizer using the class method
    loaded_tokenizer = tokenizer.__class__.load_karpathy_format(f"{file_prefix}.model")
    
    # Encode test text with both tokenizers
    original_tokens = tokenizer.encode(test_text, allowed_special=allowed_special) if hasattr(tokenizer, 'special_tokens') else tokenizer.encode(test_text)
    loaded_tokens = loaded_tokenizer.encode(test_text, allowed_special=allowed_special) if hasattr(loaded_tokenizer, 'special_tokens') else loaded_tokenizer.encode(test_text)
    
    # Compare results
    tokens_match = original_tokens == loaded_tokens
    
    # Check decoding
    original_decoded = tokenizer.decode(original_tokens)
    loaded_decoded = loaded_tokenizer.decode(loaded_tokens)
    decoding_match = original_decoded == loaded_decoded
    roundtrip_match = original_decoded == test_text
    
    # Print results
    print(f"Testing {tokenizer_name}:")
    print(f"  Original tokens: {original_tokens[:10]}{'...' if len(original_tokens) > 10 else ''}")
    print(f"  Loaded tokens:   {loaded_tokens[:10]}{'...' if len(loaded_tokens) > 10 else ''}")
    print(f"  Tokens match: {tokens_match}")
    print(f"  Decoding match: {decoding_match}")
    print(f"  Roundtrip match: {roundtrip_match}")
    
    # Check vocabulary size
    print(f"  Original vocab size: {tokenizer.vocab_size}")
    print(f"  Loaded vocab size: {loaded_tokenizer.vocab_size}")
    
    # Check any special attributes
    for attr in ['pattern', 'special_tokens']:
        if hasattr(tokenizer, attr):
            original_value = getattr(tokenizer, attr)
            loaded_value = getattr(loaded_tokenizer, attr)
            print(f"  {attr} match: {original_value == loaded_value}")
    
    # Clean up test files
    for ext in ['.model', '.vocab']:
        file_path = f"{file_prefix}{ext}"
        if os.path.exists(file_path):
            os.remove(file_path)
            
    return tokens_match and decoding_match

# Test text with special characters and multilingual content
test_text = """
This is a test of the save/load functionality.
It includes special characters: !@#$%^&*()_+
And multilingual content: こんにちは, Привет, مرحبا, 你好
As well as emojis: 😊 🚀 💻
"""

# Test with special tokens
special_test_text = "Hello <|endoftext|> world <|fim_prefix|> test"

# Test the basic tokenizer
print("\nTesting Basic Tokenizer:")
basic_tokenizer = Tokenizer()
basic_tokenizer.train(test_text, vocab_size=300, verbose=False)
test_save_load_karpathy_format(basic_tokenizer, test_text)

# Test the regex tokenizer
print("\nTesting Regex Tokenizer:")
regex_tokenizer = RegexTokenizer(pattern=GPT2_SPLIT_PATTERN)
regex_tokenizer.train(test_text, vocab_size=300, verbose=False)
test_save_load_karpathy_format(regex_tokenizer, test_text)

# Test the special tokens tokenizer
print("\nTesting Special Tokens Tokenizer:")
special_tokenizer = SpecialTokensTokenizer(pattern=GPT4_SPLIT_PATTERN)
special_tokenizer.train(test_text, vocab_size=300, verbose=False)
special_tokenizer.register_special_tokens({
    '<|endoftext|>': 100257,
    '<|fim_prefix|>': 100258
})
test_save_load_karpathy_format(special_tokenizer, special_test_text, allowed_special="all")

# Test the GPT4 tokenizer
print("\nTesting GPT4 Tokenizer:")
gpt4_tokenizer = GPT4Tokenizer()
gpt4_tokenizer.train(test_text, vocab_size=300, verbose=False)
test_save_load_karpathy_format(gpt4_tokenizer, special_test_text, allowed_special="all")

<cell_type>markdown</cell_type>## Comprehensive Performance Benchmarking

To understand the real-world performance of our tokenizer implementations, let's conduct more thorough benchmarking across a variety of text types and compare the different tokenizer classes.

In [ ]:
import time
import matplotlib.pyplot as plt
import numpy as np

def benchmark_tokenizers(test_cases, tokenizers):
    """
    Benchmark different tokenizer implementations across various test cases.
    
    Args:
        test_cases: Dictionary mapping test case names to text samples
        tokenizers: Dictionary mapping tokenizer names to tokenizer instances
        
    Returns:
        Dictionary of benchmark results
    """
    results = {}
    
    # For each tokenizer
    for tokenizer_name, tokenizer in tokenizers.items():
        tokenizer_results = {}
        
        # For each test case
        for case_name, text in test_cases.items():
            # Special token handling for applicable tokenizers
            allowed_special = "all" if hasattr(tokenizer, 'special_tokens') else None
            
            # Benchmark encoding
            start_time = time.time()
            tokens = tokenizer.encode(text, allowed_special=allowed_special) if allowed_special else tokenizer.encode(text)
            encode_time = time.time() - start_time
            
            # Benchmark decoding
            start_time = time.time()
            decoded = tokenizer.decode(tokens)
            decode_time = time.time() - start_time
            
            # Calculate metrics
            num_chars = len(text)
            num_tokens = len(tokens)
            chars_per_token = num_chars / num_tokens if num_tokens > 0 else 0
            tokens_per_second = num_tokens / encode_time if encode_time > 0 else 0
            chars_per_second = num_chars / encode_time if encode_time > 0 else 0
            roundtrip_success = text == decoded
            
            # Store results
            tokenizer_results[case_name] = {
                "chars": num_chars,
                "tokens": num_tokens,
                "encode_time": encode_time,
                "decode_time": decode_time,
                "chars_per_token": chars_per_token,
                "tokens_per_second": tokens_per_second,
                "chars_per_second": chars_per_second,
                "roundtrip_success": roundtrip_success
            }
            
        results[tokenizer_name] = tokenizer_results
    
    return results

def plot_benchmark_results(results):
    """Plot benchmark results for visual comparison."""
    tokenizer_names = list(results.keys())
    case_names = list(results[tokenizer_names[0]].keys())
    metrics = ["tokens_per_second", "chars_per_token", "chars_per_second"]
    
    fig, axes = plt.subplots(len(metrics), 1, figsize=(10, 12))
    
    for i, metric in enumerate(metrics):
        ax = axes[i]
        
        # Prepare data for plotting
        x = np.arange(len(case_names))
        width = 0.8 / len(tokenizer_names)
        
        # Plot bars for each tokenizer
        for j, tokenizer_name in enumerate(tokenizer_names):
            metric_values = [results[tokenizer_name][case][metric] for case in case_names]
            offset = j * width - (len(tokenizer_names) - 1) * width / 2
            ax.bar(x + offset, metric_values, width, label=tokenizer_name)
        
        # Set labels and title
        ax.set_ylabel(metric.replace('_', ' ').title())
        ax.set_title(f"{metric.replace('_', ' ').title()} by Test Case")
        ax.set_xticks(x)
        ax.set_xticklabels(case_names, rotation=45, ha='right')
        ax.legend()
    
    plt.tight_layout()
    plt.show()

# Define comprehensive test cases
comprehensive_test_cases = {
    "English": "The quick brown fox jumps over the lazy dog. This sentence contains all letters in the English alphabet.",
    "Code": "def fibonacci(n):\n    a, b = 0, 1\n    for _ in range(n):\n        a, b = b, a + b\n    return a\n\nprint(fibonacci(10))",
    "JSON": """{"name": "John Doe", "age": 30, "isActive": true, "hobbies": ["reading", "swimming", "cycling"]}""",
    "Emoji": "I love coding! 😊 💻 🚀 🔥 👍 🎉 💡 📱 🌎 🔍",
    "Multilingual": "Hello (English), Bonjour (French), こんにちは (Japanese), 你好 (Chinese), مرحبا (Arabic), Привет (Russian)",
    "Numbers": "0123456789 3.14159 2.71828 1234567890 9876543210 42 7 365 24 60 60",
    "Special": "<|endoftext|>This contains special tokens<|fim_prefix|>that need<|fim_middle|>special handling<|fim_suffix|>",
    "Mixed": "This is a mixed text with code: `print('hello')`, numbers: 12345, and emoji: 😊!"
}

# Create instances of each tokenizer type using the same training text
combined_training_text = " ".join(comprehensive_test_cases.values())

tokenizer_instances = {}

# Basic tokenizer
basic_tokenizer = Tokenizer()
basic_tokenizer.train(combined_training_text, vocab_size=500, verbose=False)
tokenizer_instances["Basic"] = basic_tokenizer

# Regex tokenizer
regex_tokenizer = RegexTokenizer(pattern=GPT2_SPLIT_PATTERN)
regex_tokenizer.train(combined_training_text, vocab_size=500, verbose=False)
tokenizer_instances["Regex"] = regex_tokenizer

# Special tokens tokenizer
special_tokenizer = SpecialTokensTokenizer(pattern=GPT4_SPLIT_PATTERN)
special_tokenizer.train(combined_training_text, vocab_size=500, verbose=False)
special_tokenizer.register_special_tokens({
    '<|endoftext|>': 100257,
    '<|fim_prefix|>': 100258,
    '<|fim_middle|>': 100259,
    '<|fim_suffix|>': 100260
})
tokenizer_instances["SpecialTokens"] = special_tokenizer

# GPT4 tokenizer
gpt4_tokenizer = GPT4Tokenizer()
gpt4_tokenizer.train(combined_training_text, vocab_size=500, verbose=False)
tokenizer_instances["GPT4"] = gpt4_tokenizer

# Run the benchmarks
benchmark_results = benchmark_tokenizers(comprehensive_test_cases, tokenizer_instances)

# Display results in tabular format
headers = ["Tokenizer", "Test Case", "Chars", "Tokens", "Chars/Token", "Tokens/sec", "Chars/sec", "Success"]
rows = []

for tokenizer_name, test_cases in benchmark_results.items():
    for case_name, metrics in test_cases.items():
        rows.append([
            tokenizer_name,
            case_name,
            metrics["chars"],
            metrics["tokens"],
            f"{metrics['chars_per_token']:.2f}",
            f"{metrics['tokens_per_second']:.2f}",
            f"{metrics['chars_per_second']:.2f}",
            "✓" if metrics["roundtrip_success"] else "✗"
        ])

# Print the table
print(f"{headers[0]:<15} | {headers[1]:<15} | {headers[2]:<8} | {headers[3]:<8} | "
      f"{headers[4]:<12} | {headers[5]:<12} | {headers[6]:<12} | {headers[7]}")
print("-" * 90)

for row in rows:
    print(f"{row[0]:<15} | {row[1]:<15} | {row[2]:<8} | {row[3]:<8} | "
          f"{row[4]:<12} | {row[5]:<12} | {row[6]:<12} | {row[7]}")

# Plot the results
plot_benchmark_results(benchmark_results)

<cell_type>markdown</cell_type>## Edge Case Testing for Unicode and Multilingual Content

Tokenizers often face challenges with edge cases, particularly around Unicode handling and multilingual text. Let's test our implementation against various challenging scenarios to ensure robustness.

In [ ]:
def test_edge_cases(tokenizer, test_cases, verbose=True):
    """
    Test a tokenizer against edge cases, focusing on robust handling of challenging inputs.
    
    Args:
        tokenizer: The tokenizer to test
        test_cases: Dictionary mapping test case names to text samples
        verbose: Whether to print detailed results
        
    Returns:
        Dictionary of results for each test case
    """
    results = {}
    
    for case_name, text in test_cases.items():
        if verbose:
            print(f"\n== Testing: {case_name} ==")
            print(f"Input: '{text}'")
        
        # Handle special tokens if applicable
        allowed_special = "all" if hasattr(tokenizer, 'special_tokens') else None
        
        try:
            # Try to encode
            tokens = tokenizer.encode(text, allowed_special=allowed_special) if allowed_special else tokenizer.encode(text)
            
            if verbose:
                print(f"Encoded: {tokens}")
            
            # Try to decode
            decoded = tokenizer.decode(tokens)
            
            if verbose:
                print(f"Decoded: '{decoded}'")
            
            # Check roundtrip success
            success = text == decoded
            
            if verbose:
                print(f"Roundtrip success: {success}")
                
                # If failed, show difference
                if not success:
                    print("Difference:")
                    for i, (a, b) in enumerate(zip(text, decoded)):
                        if a != b:
                            print(f"  Position {i}: '{a}' (U+{ord(a):04X}) vs '{b}' (U+{ord(b):04X})")
                            break
                    
                    # Check if lengths differ
                    if len(text) != len(decoded):
                        print(f"  Length mismatch: original={len(text)}, decoded={len(decoded)}")
            
            results[case_name] = {
                "success": success,
                "tokens": tokens,
                "decoded": decoded,
                "error": None
            }
            
        except Exception as e:
            if verbose:
                print(f"Error: {str(e)}")
            
            results[case_name] = {
                "success": False,
                "tokens": None,
                "decoded": None,
                "error": str(e)
            }
    
    # Print summary
    successes = sum(1 for r in results.values() if r["success"])
    
    if verbose:
        print(f"\nSummary: {successes}/{len(test_cases)} tests passed")
    
    return results

# Define comprehensive edge cases
unicode_edge_cases = {
    "Zero-width characters": "Text with zero-width joiner: a\u200Db and zero-width non-joiner: a\u200Cb",
    "Combining marks": "Combining diacritical marks: e\u0301 a\u0300 o\u0308",
    "RTL text": "Right-to-left text with control chars: \u2067Hello\u2069 \u2066العربية\u2069",
    "Surrogate pairs": "Surrogate pairs: \ud83d\ude00 \ud83d\ude42 \ud83d\ude1c",
    "Rare Unicode": "Rare characters: \u16A0 \u16A1 \u16A2 (Runic), \u1100 \u1101 \u1102 (Hangul)",
    "ASCII control chars": "ASCII control: \u0001\u0002\u0003\u0004\u0005\u0006\u0007",
    "Emoji sequencing": "👨‍👩‍👧‍👦 👨‍👨‍👧‍👧 👩‍👩‍👧‍👦 (family emojis with ZWJ sequences)",
    "Emoji modifiers": "👋🏻 👋🏼 👋🏽 👋🏾 👋🏿 (hand with skin tone modifiers)",
    "Language mixing": "Mixed: English, Русский, 中文, العربية, हिन्दी in one string",
    "Special tokens within Unicode": "Before<|endoftext|>اللغة العربية<|fim_prefix|>中文",
    "Very long string": "a" * 1000 + "😊" * 100 + "b" * 1000,
    "Empty string": "",
    "Boundary chars": "\u0000\u0001\uFFFF\uFFFE",
    "Bidirectional text": "Mixed English and العربية with ⁧Arabic numerals ١٢٣⁩",
    "Invalid UTF-8 replacement": "This text has invalid bytes that would be replacement chars in UTF-8 decoding"
}

# Test with our most advanced tokenizer
print("Testing GPT4Tokenizer against Unicode edge cases:")
edge_case_results = test_edge_cases(gpt4_tokenizer, unicode_edge_cases)

# Evaluate which categories cause the most issues
issue_categories = [case for case, result in edge_case_results.items() if not result["success"]]

if issue_categories:
    print("\nCategories with issues:")
    for category in issue_categories:
        print(f"- {category}")
        
    print("\nPotential improvement areas:")
    if any("RTL" in cat or "Bidirectional" in cat for cat in issue_categories):
        print("- Improve bidirectional text handling")
    if any("surrogate" in cat.lower() or "emoji" in cat.lower() for cat in issue_categories):
        print("- Enhance emoji and surrogate pair handling")
    if any("special" in cat.lower() for cat in issue_categories):
        print("- Review special token interaction with Unicode")
else:
    print("\nAll edge cases passed successfully!")